In [1]:
import pandas as pd
import requests
import io
import numpy as np
    
# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/ravikanthranjith/BITS-DM-ASSIGNMENT/master/GeneData.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df_gene = pd.read_csv(io.StringIO(download.decode('utf-8')))

#Replacing the '-' values with NaN
df_gene=df_gene.replace('-', np.nan)
#.astype("object")

#Getting Percentages of the Missing values in the respective columns
total = df_gene.isnull().sum().sort_values(ascending=False)
percent_1 = df_gene.isnull().sum()/df_gene.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(10)

#Converting all the Category columns
df_gene["Class"] = df_gene["Class"].astype('category') # This is the Prediction class
df_gene["Colour"] = df_gene["Colour"].astype('category')#Colour 
df_gene["type_of_gene"] = df_gene["type_of_gene"].astype('category')#Type of Gene
df_gene["Cell_Cycle_Cat"] = df_gene["Cell_Cycle_Cat"].astype('category')#Cell_Cycle_Cat
df_gene["chromosome"] = df_gene["chromosome"].astype('category')#chromosome

#Creation of "_Cat" Columns based on the columns converted as categories
df_gene["Class_Cat"] = df_gene["Class"].cat.codes
df_gene["Colour_Cat"] = df_gene["Colour"].cat.codes
df_gene["type_of_gene_Cat"] = df_gene["type_of_gene"].cat.codes
df_gene["Cell_Cycle_Category"] = df_gene["Cell_Cycle_Cat"].cat.codes#ell_Cycle_Cat
df_gene["chromosome_Cat"] = df_gene["chromosome"].cat.codes#chromosome


#Importing the scikit for the classifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

#Considering only the required columns Names
feature_col_names = ['Cell_Cycle_Category','Cell_Cycle_Number','chromosome_Cat','Colour_Cat','Gene_Induction','type_of_gene_Cat']

#Defining the X and Y variables
X=df_gene[feature_col_names]
Y=df_gene.Class_Cat

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1) # 70% training and 30% test

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8441666666666666


In [2]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_col_names,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('GenePrediction.png')
Image(graph.create_png())

InvocationException: GraphViz's executables not found